**Tutorial 2**

**Before running this code:** Make sure you have the following Julia packages installed: `InteractiveIsing`, `Plots`, `GLMakie`, `FileIO`.

**Install via Julia's package manager**

```julia
using Pkg
Pkg.add(url="https://github.com/rug-minds/InteractiveIsing.jl")
Pkg.add("Plots")
Pkg.add("GLMakie")
Pkg.add("FileIO")
Pkg.add("CairoMakie")
```

**What this tutorial covers:**
This tutorial demonstrates advanced features of InteractiveIsing, including:
- Custom weight functions for different coupling patterns
- Spin glass systems with random and structured disorder

**Prerequisites:**
Please ensure you have completed Tutorial 1 to understand the basic concepts of Ising simulations and hysteresis loops.

In [ ]:
## Utility functions for experiments

function newmakie(makietype, args...)
    f = makietype(args...)
    scr = GLMakie.Screen()
    display(scr, f)
    f
end



## Partition utilities

# Random partition of n into 'parts' segments (returns cumulative sizes)
function random_partition(n, parts)
    if parts == 0
        return [n]
    end
    cuts = sort(rand(1:n-1, parts-1))
    partition = [cuts[1]; diff(cuts); n - cuts[end]]
    return cumsum(partition)
end

# Uniform partition of n into 'parts' nearly equal segments (returns cumulative sizes)
function uniform_partition(n::Integer, parts::Integer)
    if parts == 0
        return [n]
    end
    base = div(n, parts)       # base size of each part
    rem = mod(n, parts)        # remainder to distribute
    partition = [i <= rem ? base + 1 : base for i in 1:parts]
    return cumsum(partition)
end

# Generate random numbers around x with range ±y
function randAround(x::Number, y::Number, dims::Int...)
    x = float(x)
    y = float(y)
    return x .+ y .* (2 .* rand(dims...) .- 1)
end


##################################################################################
### Pulse type: TrianglePulseA (simple four-segment triangular waveform)
struct TrianglePulseA end

function Processes.prepare(::TrianglePulseA, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    first  = LinRange(0, amp, round(Int,rise_point))
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    fourth = LinRange(-amp, 0, round(Int,rise_point))
    pulse = vcat(first, second, third, fourth)
    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        "Wrong length"
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::TrianglePulseA)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


##################################################################################
### Pulse type: PUNDa (no delay time between pulses)
struct PUNDa end

function Processes.prepare(::PUNDa, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    first  = LinRange(0, amp, round(Int,rise_point))
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    fourth = LinRange(-amp, 0, round(Int,rise_point))
    pulse = vcat(first, second, first, second, third, fourth, third, fourth)
    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        println("Pulse is longer than lifetime")
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    all_Es = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::PUNDa)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


##################################################################################
### Pulse type: PUNDb (with delay time between each pulse)
struct PUNDb end

function Processes.prepare(::PUNDb, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    delay  = zeros(Int, rise_point)
    first  = LinRange(0, amp, round(Int,rise_point))
    amp1   = fill(amp, rise_point)
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    amp2   = fill(-amp, rise_point)
    fourth = LinRange(-amp, 0, round(Int,rise_point))

    pulse = vcat(delay, first, amp1, second,
                 delay, first, amp1, second,
                 delay, third, amp2, fourth,
                 delay, third, amp2, fourth)

    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        println("Pulse is longer than lifetime")
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    all_Es = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::PUNDb)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


In [ ]:
# Setup: Load packages and create basic 3D Ising graph
# This follows the same pattern as Tutorial 1

using InteractiveIsing, Plots, GLMakie, FileIO
using InteractiveIsing.Processes
import InteractiveIsing as II

# Initialize a 3D Ising graph with specified dimensions and periodic boundary conditions
xL = 100  # Length in the x-dimension
yL = 100  # Length in the y-dimension
zL = 10   # Length in the z-dimension (thin film geometry)

**Custom Weight Functions**

Weight functions determine the coupling strength between neighboring spins. They are the core of defining different interaction patterns in the Ising model.

**How Weight Functions Work:**
- Input: neighbor offset `(dx, dy, dz)` and scaling parameters `(rx, ry, rz)`
- Output: coupling strength `J_ijk` between spins i, j and k
- Formula typically: `prefactor / distance_function`
- The `@WG` macro converts these into weight generators for `genAdj!()`

**Understanding `dr2` - The Distance Function:**

`dr2` represents the **squared distance** between two dipoles:
```julia
dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
```

**What this means:**
- `(dx, dy, dz)`: relative displacement in lattice units (e.g., dx=1 means "one step right")
- `(rx, ry, rz)`: scaling factors that convert lattice units to physical distance
- `dr2`: actual squared physical distance between the dipoles

**Physical Background - The Exact Dipole-Dipole Interaction:**

The **complete dipole-dipole interaction energy** is:
$$U = \frac{1}{4\pi\varepsilon_0} \cdot \frac{\mathbf{p_1} \cdot \mathbf{p_2} - 3(\mathbf{p_1} \cdot \hat{\mathbf{r}})(\mathbf{p_2} \cdot \hat{\mathbf{r}})}{r^3}$$

Where:
- **$\mathbf{p_1}, \mathbf{p_2}$**: dipole moment vectors
- **$\mathbf{r}$**: displacement vector between dipoles  
- **$\hat{\mathbf{r}} = \mathbf{r}/r$**: unit vector along displacement
- **$r = |\mathbf{r}|$**: distance between dipoles

**Key Features of This Formula:**
1. **Distance dependence**: ∝ 1/r³ (你说得完全对！)
2. **Angular dependence**: Complex function of dipole orientations and relative position
3. **Anisotropic**: Interaction strength depends on direction

**Why Do We Simplify to `1/dr2` (i.e., 1/r²)?**

**1. Ising Model Assumptions:**
   - All dipoles are **constrained to point along one axis** (e.g., z-direction)
   - This eliminates the complex angular terms: $(\mathbf{p_1} \cdot \hat{\mathbf{r}})(\mathbf{p_2} \cdot \hat{\mathbf{r}})$
   - We're left with a **simplified scalar interaction**

**2. Effective Interaction Approximation:**
   For aligned dipoles on a lattice, the effective interaction includes:
   - Direct dipole-dipole forces
   - Local field corrections
   - Lattice screening effects
   - The combined effect can be approximated with different power laws

**3. Computational Practicality:**
   - 1/r² is more stable numerically than 1/r³
   - Avoids convergence issues in Monte Carlo simulations
   - Still captures the essential **distance-dependent decay**

**4. Phenomenological Modeling:**
   - We're building an **effective model** for ferroelectric switching
   - The exact molecular-level interactions are averaged out
   - What matters is **relative strength ranking**: nearest > next-nearest > distant

**Physical Meaning in Our Model:**
- **Closer dipoles** → smaller `dr2` → stronger coupling (larger `J_ij`)
- **Farther dipoles** → larger `dr2` → weaker coupling (smaller `J_ij`)
- **Anisotropic scaling** (rx ≠ ry ≠ rz) → different interaction strengths along different axes

**Basic Weight Functions:**
- `weightfunc1(dx, dy, dz, rx, ry, rz)`: Standard ferromagnetic coupling (all positive)
  - `dr2 = (rx*dx)² + (ry*dy)² + (rz*dz)²`
  - Returns: `+1/dr2` (positive coupling, favors alignment)
  
- `weightfunc2(dx, dy, dz, rx, ry, rz)`: Antiferromagnetic checkerboard pattern  
  - Same `dr2` calculation
  - Returns: `±1/dr2` (alternating sign based on coordinate parity)

**Example Values:**
- Nearest neighbor (dx=1, dy=0, dz=0, rx=ry=rz=1): `dr2 = 1`, coupling = `1/1 = 1.0`
- Diagonal neighbor (dx=1, dy=1, dz=0, rx=ry=rz=1): `dr2 = 2`, coupling = `1/2 = 0.5`  
- Next-nearest (dx=2, dy=0, dz=0, rx=ry=rz=1): `dr2 = 4`, coupling = `1/4 = 0.25`

**Bottom Line**: The exact dipole-dipole interaction you showed (∝ 1/r³) is the fundamental physics. Our 1/r² model is a computationally practical approximation that captures the essential distance-dependent behavior for aligned dipoles in a ferroelectric lattice.

In [ ]:
# Weight function variant 1
function weightfunc1(dx, dy, dz, rx, ry, rz)
    prefac = 1
    if (abs(dx) + abs(dy)) % 2 != 0
        prefac = 1
    else
        prefac = 1
    end
    dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
    return prefac / dr2
end

# Weight function variant 2
function weightfunc2(dx, dy, dz, rx, ry, rz)
    prefac = 1
    if (abs(dx) + abs(dy)) % 2 == 0
        prefac = -1
    else
        prefac = 1
    end
    dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
    return prefac / dr2
end

**Advanced Weight Functions: Spin Glass Systems**

In this section, we explore more complex interaction patterns that create spin glass behavior and domain structures.

**Understanding Weight Function Parameters:**

Weight functions take 6 parameters: `(dx, dy, dz, x, y, z)`

- **`(dx, dy, dz)`**: **Relative neighbor offset** from the central dipole
  - These are the displacement vectors to find neighbors  
  - Example: `dx=1, dy=0, dz=0` means the neighbor is one step to the right
  - Used to determine the geometric relationship between dipoles
  
- **`(x, y, z)`**: **Absolute lattice coordinates** of the central dipole
  - These are the actual position indices in the lattice  
  - Example: `x=50, y=30, z=5` means we're at lattice site (50,30,5)
  - Used to determine which domain/region this dipole belongs to

**Domain Partitioning Tools:**
- `random_partition(n, parts)`: Randomly divides the lattice into segments
- `uniform_partition(n, parts)`: Evenly divides the lattice into uniform domains  
- `randAround(center, variation, dims)`: Creates random coefficients around a center value

**Example: `weightfunc_glass_anti()` Explained:**

```julia
function weightfunc_glass_anti(dx, dy, dz, x, y, z, x_n, y_n, z_n, coef_x, coef_y, coef_z)
    # Step 1: Find which domain the central dipole (x,y,z) belongs to
    ix = findfirst(v -> x ≤ v, x_n)  # Domain index in x-direction
    iy = findfirst(v -> y ≤ v, y_n)  # Domain index in y-direction  
    iz = findfirst(v -> z ≤ v, z_n)  # Domain index in z-direction
    
    # Step 2: Get the coupling strength for this domain
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 + (coef_z[iz] * dz)^2
    
    # Step 3: Apply antiferromagnetic checkerboard pattern
    if (abs(dx) + abs(dy)) % 2 != 0  
        prefac = -1  # Antiferromagnetic coupling
    else  
        prefac = 1   # Ferromagnetic coupling
    end
    
    return prefac / dr2
end
```

**How Weight Functions Are Used:**

1. **`@WG` macro**: Converts the function into a weight generator
2. **`genAdj!(g[1], weight_generator)`**: Builds the sparse adjacency matrix
3. **For each dipole at (x,y,z)**: 
   - Check all neighbors at (x+dx, y+dy, z+dz) within `NN` range
   - Call weight function with both relative `(dx,dy,dz)` and absolute `(x,y,z)` coordinates
   - Store the returned coupling strength in the interaction matrix

**Physical Meaning:**
These functions model realistic materials where interactions vary spatially due to:
- Chemical disorder (different domains have different properties)
- Structural defects (local variations in coupling strength)
- Phase separation (ferromagnetic vs antiferromagnetic regions)
- Interface effects (coupling changes near boundaries)

**Detailed Breakdown of Spin Glass Weight Functions**

Let's understand exactly how these weight functions work:

**Key Concepts:**
- **`(dx, dy, dz)`**: Relative displacement to find neighbors (e.g., dx=1 means "neighbor to the right")
- **`(x, y, z)`**: Absolute position of the central dipole in the lattice
- **Domain assignment**: Each dipole belongs to a domain based on its absolute position `(x, y, z)`

**Step-by-Step Process:**

1. **Domain Identification** (using absolute coordinates):
   ```julia
   ix = findfirst(v -> x ≤ v, x_n)  # Which x-domain contains this dipole?
   iy = findfirst(v -> y ≤ v, y_n)  # Which y-domain contains this dipole?
   iz = findfirst(v -> z ≤ v, z_n)  # Which z-domain contains this dipole?
   ```

2. **Distance Calculation** (using relative coordinates):
   ```julia
   dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 + (coef_z[iz] * dz)^2
   ```
   
   **What `dr2` represents here:**
   - **Domain-dependent scaling**: Each domain has different `coef_x[ix], coef_y[iy], coef_z[iz]` values
   - **Physical interpretation**: Different domains have different "effective lattice spacings"
   - **Example**: 
     - Domain 1: `coef_x = 1.2` → interactions are stretched (weaker)
     - Domain 2: `coef_x = 0.8` → interactions are compressed (stronger)
   - **Result**: `dr2` varies spatially, creating disorder in coupling strengths

3. **Interaction Type** (ferromagnetic vs antiferromagnetic):
   - **`weightfunc_glass_ferro()`**: Always `+1/dr2` (ferromagnetic)
   - **`weightfunc_glass_anti()`**: Alternating `±1/dr2` based on `(dx, dy)` parity
   - **`weightfunc_AntiWithFerro()`**: Domain-dependent interaction type

**Why This Creates Spin Glass Behavior:**

The combination of:
- **Spatial disorder** in `dr2` values (different domains have different coupling strengths)
- **Competing interactions** (ferro vs antiferromagnetic regions)
- **Distance-dependent decay** (`1/dr2` falloff)

Creates a **frustrated energy landscape** where:
- No single spin configuration can minimize all interactions simultaneously
- Multiple local energy minima exist (spin glass states)
- System shows complex, history-dependent behavior

**Example Usage:**
- For dipole at position `(50, 30, 5)` looking at neighbor `(51, 30, 5)`:
  - `(x, y, z) = (50, 30, 5)` → determines domain coefficients: `coef_x[ix], coef_y[iy], coef_z[iz]`
  - `(dx, dy, dz) = (1, 0, 0)` → `dr2 = (coef_x[ix] * 1)² = coef_x[ix]²`
  - Final coupling: `±1/coef_x[ix]²` (sign depends on interaction type)
  
**Result**: Each domain has different coupling properties, creating complex energy landscapes typical of spin glasses where `dr2` varies spatially to model realistic material disorder.

In [ ]:
x_n = random_partition(xL, 20)
y_n = random_partition(yL, 20)
z_n = random_partition(zL, 3)
coef_x =randAround(1, 0.2, 20)
coef_y =randAround(1, 0.2, 20)
coef_z =randAround(1, 0.2, 3)
println(x_n)
println(y_n)
println(z_n)
println(coef_x)
println(coef_y)
println(coef_z)


function weightfunc_glass_anti(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x,coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    if (abs(dx) + abs(dy)) % 2 != 0  
        prefac = -1
    else  
        prefac = 1
    end
    return prefac / dr2
end

function weightfunc_glass_ferro(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x,coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    prefac = 1
    return prefac / dr2
end

function weightfunc_AntiWithFerro(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x, coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    if ix % 2 == 0 && iy %2==0
        prefac=1
    elseif xor(isodd(ix), isodd(iy))
        if (abs(dx) + abs(dy)) % 2 != 0  
            prefac = -1
        else  
            prefac = 1
        end
    end
    return prefac / dr2
end

wg_glass1 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_glass_ferro(dx,dy,dz,x,y,z,x_n, y_n, z_n, coef_x,coef_y,coef_z)" NN = (2,2,2)
wg_glass2 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_glass_anti(dx,dy,dz,x,y,z, x_n, y_n, z_n, coef_x,coef_y,coef_z)" NN = (2,2,2)
wg_glass3 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_AntiWithFerro(dx, dy, dz, x, y, z, x_n, y_n, z_n, coef_x, coef_y, coef_z)" NN = (2,2,2)


In [ ]:
# Example 1: normal weight function (ferro)
g = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g.hamiltonian = Ising(g) + DepolField(g, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g.hamiltonian = sethomogenousparam(g.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.

homogeneousself!(g, -1)

### We set Temperature to 1.5
settemp(g,1.5)

### Change nearest neighbours
####################################################################################
wg1 = @WG "(dx,dy,dz,x,y,z) -> weightfunc1(dx,dy,dz,1,1,1)" NN = (2,2,2)
####################################################################################
genAdj!(g[1], wg1)


fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)
### estimate time
est_remaining(process(g))

# Wait until it is done
args = process(g) |> fetch # If you want to close ctr+c
# args = process(g) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w2=newmakie(lines, voltage, Pr)
w3=newmakie(lines,Pr)


In [ ]:
# Example 2：spin glass anti alignment
g_glass_anti = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g_glass_anti)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g_glass_anti.hamiltonian = Ising(g_glass_anti) + DepolField(g_glass_anti, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g_glass_anti.hamiltonian = sethomogenousparam(g_glass_anti.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.
homogeneousself!(g_glass_anti, -1)

### We set Temperature to 1.5
settemp(g_glass_anti,1.5)


genAdj!(g_glass_anti[1], wg_glass2)

fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g_glass_anti, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)

### estimate time
est_remaining(process(g_glass_anti))

# Wait until it is done
args = process(g_glass_anti) |> fetch # If you want to close ctr+c
# args = process(g_glass_anti) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w_anti_2=newmakie(lines, voltage, Pr)
w_anti_3=newmakie(lines,Pr)


In [ ]:
# Example 3：spin glass with mixed interactions
g_GlassMix = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g_GlassMix)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g_GlassMix.hamiltonian = Ising(g_GlassMix) + DepolField(g_GlassMix, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g_GlassMix.hamiltonian = sethomogenousparam(g_GlassMix.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.
homogeneousself!(g_GlassMix, -1)

### We set Temperature to 1.5
settemp(g_GlassMix,1.5)


genAdj!(g_GlassMix[1], wg_glass3)

fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g_GlassMix, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)

### estimate time
est_remaining(process(g_GlassMix))

# Wait until it is done
args = process(g_GlassMix) |> fetch # If you want to close ctr+c
# args = process(g_GlassMix) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

w_Mix_2=newmakie(lines, voltage, Pr)
w_Mix_3=newmakie(lines,Pr)